In [2]:
import torchvision
from torchvision import transforms
import torch
from torch import nn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                    mean=[0.4190, 0.4196, 0.4361],
                                    std=[0.2553, 0.1909, 0.1646],
                                )])
dataset = torchvision.datasets.ImageFolder("/content/drive/MyDrive/Fish_Dataset/", transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=200, shuffle=True, num_workers = 2)

Run the cell below to get mean and std, but it's already been calculated. 
mean = tensor([0.4190, 0.4196, 0.4361])

std = tensor([0.2553, 0.1909, 0.1646])

In [10]:
nimages = 0
mean = 0.0
var = 0.0
for i_batch, batch_target in enumerate(dataloader):
    batch = batch_target[0]
    # Rearrange batch to be the shape of [B, C, W * H]
    batch = batch.view(batch.size(0), batch.size(1), -1)
    # Update total number of images
    nimages += batch.size(0)
    # Compute mean and std here
    mean += batch.mean(2).sum(0) 
    var += batch.var(2).sum(0)
    print("Doing: " + str(i_batch))

mean /= nimages
var /= nimages
std = torch.sqrt(var)

print(mean)
print(std)

Doing: 0
Doing: 1
Doing: 2
Doing: 3
Doing: 4
Doing: 5
Doing: 6
Doing: 7
Doing: 8
Doing: 9
Doing: 10
Doing: 11
Doing: 12
Doing: 13
Doing: 14
Doing: 15
Doing: 16
Doing: 17
Doing: 18
Doing: 19
Doing: 20
Doing: 21
Doing: 22
Doing: 23
Doing: 24
Doing: 25
Doing: 26
Doing: 27
Doing: 28
Doing: 29
Doing: 30
Doing: 31
Doing: 32
Doing: 33
Doing: 34
Doing: 35
Doing: 36
Doing: 37
Doing: 38
Doing: 39
Doing: 40
Doing: 41
Doing: 42
Doing: 43
Doing: 44
tensor([0.4190, 0.4196, 0.4361])
tensor([0.2553, 0.1909, 0.1646])


In [21]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()

    self.total_classes = 9

    self.conv = nn.Sequential(
        nn.Conv2d(3, 10, 2),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(10, 20, 2),
        nn.ReLU(),
        nn.MaxPool2d(2),
    )

    self.dense = nn.Sequential(
        nn.Linear(20 * 55 * 55, 50),
        nn.ReLU(),
        nn.Linear(50, 9)
    )

  
  def forward(self, input):
    x = self.conv(input)
    x = x.view(-1, 20 * 55 * 55)
    return self.dense(x)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
device

device(type='cuda')

In [23]:
def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())

In [34]:
net = Model()
net.to(device)

optimizer = torch.optim.Adadelta(net.parameters(), lr=0.01)
num_epochs = 45
loss_fn = nn.CrossEntropyLoss()

for i in range(num_epochs):
  net.train()
  running_loss = 0.00
  running_accuracy = 0.00
  for data, label in dataloader:
    data, label = data.to(device), label.to(device)
    predictions = net(data)
    optimizer.zero_grad()
    loss = loss_fn(predictions, label)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    running_accuracy += accuracy(predictions, label)
  running_loss /= len(dataloader)
  running_accuracy /= len(dataloader)
  print(f"Epoch {i}, loss = {running_loss}, accuracy = {running_accuracy}")

Epoch 0, loss = 1.9978107531865439, accuracy = 0.2996666431427002
Epoch 1, loss = 1.6869672377904257, accuracy = 0.46222221851348877
Epoch 2, loss = 1.449644480811225, accuracy = 0.5368888974189758
Epoch 3, loss = 1.2651700841055975, accuracy = 0.6084443926811218
Epoch 4, loss = 1.1359228677219815, accuracy = 0.64211106300354
Epoch 5, loss = 1.0279253774219088, accuracy = 0.6784444451332092
Epoch 6, loss = 0.9457156525717841, accuracy = 0.7029999494552612
Epoch 7, loss = 0.8880972610579596, accuracy = 0.728444516658783
Epoch 8, loss = 0.83810895813836, accuracy = 0.7385556101799011
Epoch 9, loss = 0.7807979941368103, accuracy = 0.7600000500679016
Epoch 10, loss = 0.7425917704900106, accuracy = 0.7755557894706726
Epoch 11, loss = 0.7016069518195258, accuracy = 0.7877777218818665
Epoch 12, loss = 0.6629219293594361, accuracy = 0.8046666383743286
Epoch 13, loss = 0.6301883247163561, accuracy = 0.8157777190208435
Epoch 14, loss = 0.6082153989209069, accuracy = 0.8216666579246521
Epoch 15, 